In [1]:
import csv

In [2]:
Authors = dict()   # Authors dictionary. Keys = authors. Values = list written papers' indexes.
Papers  = dict()   # Papers dictionary. Keys = papers' indexes. Values = list of its authors.

with open('neo4j.csv') as planilha:
    readCSV =csv.reader(planilha, delimiter=',')
    i = 0
    for row in readCSV:
        i += 1
        Papers[i] = row 
        for author in row:
            if author not in Authors:
                Authors[author] = [i]
            else:
                Authors[author].append(i)
                
'''IMPORTANT: I did some manual pre-processing in the neo4j.csv and the script depends on it to work. 
Basically, some small adjustments -- e.g., substituting irregular characters as ó, ü and spaces, as Cypher cannot use them to name variables'''

In [3]:
'''Create a text file containing the script that loads the Graph DB'''

script = open('script.txt', 'w')

# Create the paper nodes
for paper in Papers:
    script.write('CREATE (P{}:Paper {{id:\'{}\'}})\n'.format(paper, paper))
    
# Create the author nodes and the edges
for author in Authors:
    script.write('CREATE ({}:Author {{name:\'{}\'}})\n'.format(author, author))
    script.write('CREATE\n')
    
    for i in range(len(Authors[author])):
        authored_paper = Authors[author][i]
        if i+1 == len(Authors[author]):
            script.write('    ({})-[:AUTHORED]->(P{})\n'.format(author, authored_paper))
        else:
            script.write('    ({})-[:AUTHORED]->(P{}),\n'.format(author, authored_paper))
        
# Create author-author edges (collab) (optional)

colab = False
if colab == True:

    script.write('CREATE\n')
    
    for p in Papers:
        authors = Papers[p] # fetch p authors (list)
        for i in range(len(authors)):
            for j in range(i+1,len(authors)):
                script.write('    ({})-[:COLLABORATED]->({}),\n'.format(authors[i], authors[j]))
    

    with open("script.txt", 'rb+') as f: # workaround to delete the last comma
        f.seek(0,2)                      # end of file
        size=f.tell()                    # the size...
        f.truncate(size-2)               # truncate at that size - how ever many characters

script.close()

    

In [4]:
'''Create a collab graph (without the papers)'''
    
colab = dict()    # dictionary stores partners -- key = author; value = list of authors that collaborated with a.
for a in Authors:
    colab[a] = []

for p in Papers:
    authors = Papers[p] #fetch p authors (list)
    for a in authors:
        for b in authors:
            if a != b and not a in colab[b] and not b in colab[a]:
                colab[a].append(b)
              
script = open('colabscript.txt', 'w')
            
for author in Authors: # write the authors as variables in the cypher script
    script.write('CREATE ({}:Author {{name:\'{}\'}})\n'.format(author, author))
    

script.write('CREATE\n')

for a in colab: # write collab edges (a->b implies not b->a)
    for b in colab[a]:
        script.write('({})-[:COLLABORATED]->({}),\n'.format(a, b))
        
script.close()
            
with open("colabscript.txt", 'rb+') as f: # workaround to delete the last comma
    f.seek(0,2)                 # end of file
    size=f.tell()               # the size...
    f.truncate(size-2)          # truncate at that size - how ever many characters